# Exercise 3 - Emotion Recognition
## Convoluted Feelings


### Data
For this exercise, you will be working on the Twitter Emotion Recognition task. The goal of this task is to infer the
affectual state of a person from their tweet. You will be using the Tweeteval dataset to train your model. You can
access the related GitHub repositories from the links below.

● Tweeteval Repository: https://github.com/cardiffnlp/tweeteval

● Emotion Detection (our task): https://github.com/cardiffnlp/tweeteval/tree/main/datasets/emotion

The ability to process and load custom datasets for your projects/research is an important skill. Most tasks you will
complete as an NLP practitioner/researcher will require you to handle different data sources, formats, and files.
Therefore, we want you to figure out how to load the emotion recognition dataset from its respective repository
using all the necessary files (e.g., train_text.txt, train_labels.txt, mapping.txt) and train your model.

### Task: Emotion Recognition with a CNN
Implement an emotion recognition classifier in PyTorch or PyTorch Lightning. You can reuse the class structure
from exercise 2, which is an adaption from Rao and McMahan. However, you are free to create your own, new
class structure. Keep in mind that for emotion prediction, unlike in Exercise 1, we work on the word level instead of
the character level. Thus, your Vocabulary class (if you have one) will not hold a vocabulary of characters.
Remember to document your code with docstrings and/or comments and/or text cells.
1. Pick two different emotion classes for your model to predict (e.g., anger and joy). Load/filter your
dataset to include only the related class data. Create another dataset and change only one of the
classes (e.g., anger and sadness) this time.
2. Your goal is to find the optimal model architecture and training regime for your CNN classifier. Pick one
of the datasets you created and start experimenting. Experiment with at least three different combinations
(sets) of hyperparameters, with at least two different values each, e.g., optimizer, learning rate, dropout,
number of filters, stride, kernel size, pooling, and batch size. Report the combinations and corresponding
results (accuracy and F1-macro) on the development set in a table:
3. Use your best-performing model settings to train another model on the second dataset. Report
the model performance (accuracy and F1-macro) on the test set of both datasets.
4. What could be the reason that the specific combination/values of hyperparameters resulted in the best
model performance? Don’t worry about the exact reasoning, the goal is just to provide educated (or wellreasoned) guesses.

In [1]:
!pip install spacy

In [2]:
import os
import re
import random
import time
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import spacy
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.utils.data as data, torchvision as tv
import torch.optim as optim

In [3]:
nlp = spacy.blank('en')

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
#import os
#data_dir = "/content/drive/MyDrive/Exercise_3"

### Read data
Reading each lines in the data and it's corresponding labels below.

In [8]:
def read_text(text_path):
  with open(text_path, 'r') as file:
        text_data = file.read().splitlines()
  return text_data

def read_labels(label_path):
  with open(label_path, 'r') as file:
        label_data = file.read().splitlines()
  return label_data

## Load Train, Validation, Test data
Loading train, validation, and test dataset and labels

In [9]:
train_txt_file = "train_text.txt"
train_label_file = "train_labels.txt"
train_data = read_text(train_txt_file)
train_labels = read_labels(train_label_file)

val_txt_file = "val_text.txt"
val_label_file = "val_labels.txt"
val_data = read_text(val_txt_file)
val_labels = read_labels(val_label_file)

test_txt_file = "test_text.txt"
test_label_file = "test_labels.txt"
test_data = read_text(test_txt_file)
test_labels = read_labels(test_label_file)

### Example data
Example of how data and labels look like. Mappings are as follows:

0 ---> Anger

1 ---> Joy

2 ---> Optimism

3 ---> Sadness

In [10]:
train_data[0], train_labels[0]

("“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry ",
 '2')

In [11]:
val_data[0], val_labels[0]

('@user @user Oh, hidden revenge and anger...I rememberthe time,she rebutted you. ',
 '0')

In [12]:
test_data[0], test_labels[0]

('#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse ',
 '3')

In [13]:
train_data[0:5]

["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry ",
 "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs ",
 "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu ",
 "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight ",
 "it's pretty depressing when u hit pan on ur favourite highlighter "]

In [14]:
train_labels[0:5]

['2', '0', '1', '0', '3']

## Section 1: Emotional Recognition Model: Anger or Joy

In this section, we will choose data from anger (0) and joy (1) classes and train a Emotion Recognition model using only these two classes.

In [15]:
train_d_aj = [data for data, label in zip(train_data, train_labels) if label in ('0', '1')]
train_l_aj = [label for label in train_labels if label in ('0', '1')]

val_d_aj = [data for data, label in zip(val_data, val_labels) if label in ('0', '1')]
val_l_aj = [label for label in val_labels if label in ('0', '1')]

test_d_aj = [data for data, label in zip(test_data, test_labels) if label in ('0', '1')]
test_l_aj = [label for label in test_labels if label in ('0', '1')]

Examples of filtered data:

In [16]:
train_d_aj[0:10]

["My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs ",
 "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu ",
 "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight ",
 '@user but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant . ',
 'Tiller and breezy should do a collab album. Rapping and singing prolly be fire ',
 '@user broadband is shocking regretting signing up now #angry #shouldofgonewithvirgin ',
 '@user Look at those teef! #growl ',
 '@user @user USA was embarrassing to watch. When was the last time you guys won a game..? #horrible #joke ',
 'Your glee filled Normy dry humping of the most recent high profile celebrity break up is pathetic &amp; all that is wrong with the world today. ',
 'What a fucking muppet.  @user  #stalker. ']

In [17]:
train_l_aj[0:10]

['0', '1', '0', '0', '1', '0', '0', '0', '0', '0']

In [18]:
val_l_aj[0:10]

['0', '0', '0', '0', '0', '0', '0', '1', '0', '0']

tokenize, encode, converting to tensor functions. Reference of tokenize, encode functions: tutorial notebook

In [19]:
def tokenize(texts):
  max_len = 0
  tokenized_texts = []
  word2idx = {}

  # Add <pad> and <unk> tokens to the vocabulary
  word2idx['<pad>'] = 0
  word2idx['<unk>'] = 1

  # Building our vocab from the corpus starting from index 2
  idx = 2
  for sent in texts:
    tokenized_sent = nlp(sent)
    # Add `tokenized_sent` to `tokenized_texts`
    tokenized_texts.append(tokenized_sent)
    # Add new token to `word2idx`
    for token in tokenized_sent:
      # string any token objects are different things, be careful.
      if token.text not in word2idx:
        word2idx[token.text] = idx
        idx += 1

        # Update `max_len`
    max_len = max(max_len, len(tokenized_sent))

  return tokenized_texts, word2idx, max_len


def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to e
        tokenized_padded_sent = list(tokenized_sent) + ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(str(token)) for token in tokenized_padded_sent]
        input_ids.append(input_id)

    return np.array(input_ids), word2idx


def get_tokenized_encoded_ids(text):
  # get tokenized and encoded data input ids
  tokenized_texts, word2idx, max_len = tokenize(text)
  input_ids, word2idx = encode(tokenized_texts, word2idx, max_len)
  # converting input ids to torch.Tensor
  input_ids = torch.from_numpy(input_ids)
  return input_ids, word2idx

def get_tensor_labels(labels):
  # converting labels to torch.Tensor
  label_int = [int(item) for item in labels]
  label_tensors = torch.tensor(label_int)
  return label_tensors

In [20]:
# tokenizing, encoding, converting to tensors
train_id_aj, word2idx = get_tokenized_encoded_ids(train_d_aj)
val_id_aj, _ = get_tokenized_encoded_ids(val_d_aj)
test_id_aj, _ = get_tokenized_encoded_ids(test_d_aj)

# converting labels to tensors
train_l_aj = get_tensor_labels(train_l_aj)
val_l_aj = get_tensor_labels(val_l_aj)
test_l_aj = get_tensor_labels(test_l_aj)

Example of dataset after tokenization, encoding, and converting to tensors

In [21]:
train_id_aj[0:5]

tensor([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  9, 14, 15, 16, 17, 18,
         17, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [20, 21,  8,  6, 22, 23, 16, 24, 25, 26, 27, 28, 29, 30, 21, 31, 32, 33,
         34, 35,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [36, 37, 38, 39, 11, 40, 41, 42, 37, 43, 44, 45, 46, 47, 48, 49,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [50, 21, 51, 52, 25, 53, 54, 55, 56, 57, 22, 58, 59, 60, 61, 62, 16, 63,
         64, 60, 65, 66,  8, 51, 67, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [68, 69, 70, 71, 72, 73, 74, 75, 16, 76, 69, 77, 78, 79, 80,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  

In [22]:
train_l_aj[0:5]

tensor([0, 1, 0, 0, 1])

Create dataset and dataloader objects from tensors of data. Dataloader object will be used in training.

In [23]:
random_seed = 42
train_dataset = TensorDataset(train_id_aj, train_l_aj)
train_sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(random_seed))

val_dataset = TensorDataset(val_id_aj, val_l_aj)
test_dataset = TensorDataset(test_id_aj, test_l_aj)

batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [24]:
import torchtext.vocab as vocab
class CNN(nn.Module):
    def __init__(self, embed_dim, filter_sizes, num_filters, num_classes, dropout, pretrained_embeddings, vocab_size=len(word2idx)):
        super(CNN, self).__init__()
        if pretrained_embeddings is not None:
            print("using pretrained embedding")
            glove = vocab.GloVe(name='6B', dim=embed_dim)
            self.embedding = nn.Embedding.from_pretrained(glove.vectors, freeze=True)
        else:
            print("training embedding")
            self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=0)
        self.conv1d_list = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels=embed_dim, out_channels=num_filters[i], kernel_size=filter_sizes[i]),
                nn.BatchNorm1d(num_filters[i]),
                nn.LeakyReLU(negative_slope=0.01),
                nn.MaxPool1d(kernel_size=filter_sizes[i])
            )
            for i in range(len(filter_sizes))
        ])
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        x_embed = self.embedding(input_ids).float()
        x_reshaped = x_embed.permute(0, 2, 1)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        logits = self.fc(self.dropout(x_fc))
        return logits

In [25]:
class Train:
  def __init__(
      self,
      embed_dim=100,
      filter_sizes=[3, 4, 5],
      num_filters=[150, 150, 150],
      dropout=0.2,
      num_classes=2,
      lr=0.01,
      rho=0.95,
      epoch=5,
      pretrained_embeddings=None,
      weight_decay = 1e-5
    ):
    self._embed_dim = embed_dim
    self._filter_sizes = filter_sizes
    self._num_filters = num_filters
    self._dropout = dropout
    self._num_classes = num_classes
    self._lr = lr
    self._rho = rho
    self._epoch = epoch
    self._pretrained_embeddings = pretrained_embeddings
    self._weight_decay = weight_decay

  def train_model(self, device, train_dataloader, validation_dataloader):
    # Instantiate CNN model
    model = CNN(embed_dim=self._embed_dim,
                filter_sizes=self._filter_sizes,
                num_filters=self._num_filters,
                num_classes=self._num_classes,
                dropout=self._dropout,
                pretrained_embeddings=self._pretrained_embeddings)

    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate SDG Optimizer
    optimizer = optim.SGD(model.parameters(), lr=self._lr, weight_decay=self._weight_decay)

    # cross entropy loss function
    loss_fn = nn.CrossEntropyLoss()

    # Start training loop
    print("Start training...\n")
    best_validation_loss = float('inf')
    best_model = None

    for epoch_i in range(self._epoch):
        total_train_loss = 0
        total_validation_loss = 0
        y_true_t = []
        y_pred_t = []

        y_true_v = []
        y_pred_v = []

        # Put the model into the training mode
        model.train()
        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()
            logits = model(b_input_ids)
            loss = loss_fn(logits, b_labels)
            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            y_true_t.extend(b_labels.cpu().numpy())
            y_pred_t.extend(logits.argmax(1).cpu().numpy())

        # Calculate the average training loss
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Calculate accuracy and F1-macro for the training data
        train_accuracy = accuracy_score(y_true_t, y_pred_t)
        train_f1_macro = f1_score(y_true_t, y_pred_t, average='macro')

        # Put the model into evaluation mode
        model.eval()
        with torch.no_grad():
            for step, batch in enumerate(validation_dataloader):
                b_input_ids, b_labels = tuple(t.to(device) for t in batch)
                logits = model(b_input_ids)
                loss = loss_fn(logits, b_labels)
                total_validation_loss += loss.item()
                y_true_v.extend(b_labels.cpu().numpy())
                y_pred_v.extend(logits.argmax(1).cpu().numpy())

        # Calculate the average validation loss
        avg_validation_loss = total_validation_loss / len(validation_dataloader)

        # Calculate accuracy and F1-macro for the validation data
        val_accuracy = accuracy_score(y_true_v, y_pred_v)
        val_f1_macro = f1_score(y_true_v, y_pred_v, average='macro')

        # Save the best model based on validation loss
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            best_model = model.state_dict()

        print(f"Epoch {epoch_i + 1:2d} | Train Loss: {avg_train_loss:.6f} | Train Acc: {train_accuracy:.4f} | Train F1-Macro: {train_f1_macro:.4f}")
        print(f"Epoch {epoch_i + 1:2d} | Validation Loss: {avg_validation_loss:.6f} | Validation Acc: {val_accuracy:.4f} | Validation F1-Macro: {val_f1_macro:.4f}")
        print(".........................................................................................")

    # Load the best model for further evaluation or testing
    model.load_state_dict(best_model)
    return model

In [26]:
def test_model(model, device, test_dataloader):
  model.eval()
  y_true = []
  y_pred = []

  with torch.no_grad():
      for step, batch in enumerate(test_dataloader):
          b_input_ids, b_labels = tuple(t.to(device) for t in batch)
          logits = model(b_input_ids)
          y_true.extend(b_labels.cpu().numpy())
          y_pred.extend(logits.argmax(1).cpu().numpy())

  test_accuracy = accuracy_score(y_true, y_pred)
  test_f1_macro = f1_score(y_true, y_pred, average='macro')

  print(f"Test Accuracy: {test_accuracy:.4f}")
  print(f"Test F1-Macro: {test_f1_macro:.4f}")

In [27]:
from sklearn.metrics import f1_score, accuracy_score

# use the GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


### Experiments on different Hyperparameter

In [ ]:
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 3, 4, 5], num_filters=[100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding


.vector_cache/glove.6B.zip: 862MB [02:46, 5.19MB/s]                           
100%|█████████▉| 399999/400000 [01:14<00:00, 5360.91it/s]


Start training...

Epoch  1 | Train Loss: 0.863562 | Train Acc: 0.5280 | Train F1-Macro: 0.4963
Epoch  1 | Validation Loss: 0.684480 | Validation Acc: 0.5992 | Validation F1-Macro: 0.4496
.........................................................................................
Epoch  2 | Train Loss: 0.818041 | Train Acc: 0.5731 | Train F1-Macro: 0.4747
Epoch  2 | Validation Loss: 0.705643 | Validation Acc: 0.6031 | Validation F1-Macro: 0.4322
.........................................................................................
Epoch  3 | Train Loss: 0.798529 | Train Acc: 0.5821 | Train F1-Macro: 0.4922
Epoch  3 | Validation Loss: 0.706622 | Validation Acc: 0.6070 | Validation F1-Macro: 0.4272
.........................................................................................
Epoch  4 | Train Loss: 0.792956 | Train Acc: 0.5859 | Train F1-Macro: 0.4987
Epoch  4 | Validation Loss: 0.705339 | Validation Acc: 0.6070 | Validation F1-Macro: 0.4272
...................................

In [ ]:
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.5, weight_decay=0.00001, filter_sizes=[3, 4, 5], num_filters=[100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 1.034957 | Train Acc: 0.6067 | Train F1-Macro: 0.4946
Epoch  1 | Validation Loss: 0.634822 | Validation Acc: 0.6187 | Validation F1-Macro: 0.3916
.........................................................................................
Epoch  2 | Train Loss: 0.998742 | Train Acc: 0.5726 | Train F1-Macro: 0.4953
Epoch  2 | Validation Loss: 0.655613 | Validation Acc: 0.6070 | Validation F1-Macro: 0.3956
.........................................................................................
Epoch  3 | Train Loss: 0.994742 | Train Acc: 0.5840 | Train F1-Macro: 0.5104
Epoch  3 | Validation Loss: 0.654424 | Validation Acc: 0.5992 | Validation F1-Macro: 0.4004
.........................................................................................
Epoch  4 | Train Loss: 0.963215 | Train Acc: 0.5863 | Train F1-Macro: 0.5230
Epoch  4 | Validation Loss: 0.655782 | Validation Acc: 0.6031 | Validation F1-Macro: 0.3939
........

In [ ]:
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.001, dropout=0.2, weight_decay=0.000001, filter_sizes=[3, 3, 4, 5], num_filters=[100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 0.793977 | Train Acc: 0.5721 | Train F1-Macro: 0.4798
Epoch  1 | Validation Loss: 0.646192 | Validation Acc: 0.6226 | Validation F1-Macro: 0.4873
.........................................................................................
Epoch  2 | Train Loss: 0.747948 | Train Acc: 0.6139 | Train F1-Macro: 0.5240
Epoch  2 | Validation Loss: 0.637351 | Validation Acc: 0.6304 | Validation F1-Macro: 0.4806
.........................................................................................
Epoch  3 | Train Loss: 0.716448 | Train Acc: 0.6191 | Train F1-Macro: 0.5306
Epoch  3 | Validation Loss: 0.640571 | Validation Acc: 0.6265 | Validation F1-Macro: 0.4782
.........................................................................................
Epoch  4 | Train Loss: 0.704576 | Train Acc: 0.6314 | Train F1-Macro: 0.5458
Epoch  4 | Validation Loss: 0.639983 | Validation Acc: 0.6304 | Validation F1-Macro: 0.4806
........

In [ ]:
train_obj = Train(embed_dim=50, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 3, 4, 5], num_filters=[150, 150, 150, 150])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding


100%|█████████▉| 399999/400000 [00:16<00:00, 24436.29it/s]


Start training...

Epoch  1 | Train Loss: 0.886553 | Train Acc: 0.5266 | Train F1-Macro: 0.4964
Epoch  1 | Validation Loss: 0.626182 | Validation Acc: 0.5914 | Validation F1-Macro: 0.4869
.........................................................................................
Epoch  2 | Train Loss: 0.776680 | Train Acc: 0.5844 | Train F1-Macro: 0.4991
Epoch  2 | Validation Loss: 0.618973 | Validation Acc: 0.6148 | Validation F1-Macro: 0.4385
.........................................................................................
Epoch  3 | Train Loss: 0.791739 | Train Acc: 0.5968 | Train F1-Macro: 0.5074
Epoch  3 | Validation Loss: 0.618992 | Validation Acc: 0.6109 | Validation F1-Macro: 0.4364
.........................................................................................
Epoch  4 | Train Loss: 0.761243 | Train Acc: 0.5935 | Train F1-Macro: 0.5045
Epoch  4 | Validation Loss: 0.618871 | Validation Acc: 0.6148 | Validation F1-Macro: 0.4455
...................................

In [ ]:
train_obj = Train(embed_dim=50, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 4, 5], num_filters=[150, 150, 150])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 0.794772 | Train Acc: 0.5527 | Train F1-Macro: 0.4963
Epoch  1 | Validation Loss: 0.605483 | Validation Acc: 0.5992 | Validation F1-Macro: 0.4004
.........................................................................................
Epoch  2 | Train Loss: 0.784155 | Train Acc: 0.5821 | Train F1-Macro: 0.4936
Epoch  2 | Validation Loss: 0.618208 | Validation Acc: 0.5953 | Validation F1-Macro: 0.3904
.........................................................................................
Epoch  3 | Train Loss: 0.767031 | Train Acc: 0.5935 | Train F1-Macro: 0.4923
Epoch  3 | Validation Loss: 0.614710 | Validation Acc: 0.5992 | Validation F1-Macro: 0.4082
.........................................................................................
Epoch  4 | Train Loss: 0.778554 | Train Acc: 0.5816 | Train F1-Macro: 0.4888
Epoch  4 | Validation Loss: 0.621688 | Validation Acc: 0.6031 | Validation F1-Macro: 0.3939
........

### Best Parameter Combination

embed_dim=300

pretrained_embeddings=True

epoch=50

lr=0.0001

dropout=0.2

weight_decay=0.00001

filter_sizes=[3, 3, 4, 5]

num_filters=[100, 100, 100, 100]

## Section 2: Emotional Recognition Model: Anger or Sadness

In this section, we will choose data from anger (0) and sadness (3) classes and train a Emotion Recognition model using the best hyperparamters we received in previous section.

In [28]:
from sklearn.metrics import f1_score, accuracy_score

train_d_as = [data for data, label in zip(train_data, train_labels) if label in ('0', '3')]
train_l_as = [label for label in train_labels if label in ('0', '3')]
train_l_as = ["1" if value == "3" else value for value in train_l_as]

val_d_as = [data for data, label in zip(val_data, val_labels) if label in ('0', '3')]
val_l_as = [label for label in val_labels if label in ('0', '3')]
val_l_as = ["1" if value == "3" else value for value in val_l_as]

test_d_as = [data for data, label in zip(test_data, test_labels) if label in ('0', '3')]
test_l_as = [label for label in test_labels if label in ('0', '3')]
test_l_as = ["1" if value == "3" else value for value in test_l_as]

# tokenizing, encoding, converting to tensors
train_id_as, word2idx = get_tokenized_encoded_ids(train_d_as)
val_id_as, _ = get_tokenized_encoded_ids(val_d_as)
test_id_as, _ = get_tokenized_encoded_ids(test_d_as)

# converting labels to tensors
train_l_as = get_tensor_labels(train_l_as)
val_l_as = get_tensor_labels(val_l_as)
test_l_as = get_tensor_labels(test_l_as)

# prepare dataloader
random_seed = 42
train_dataset = TensorDataset(train_id_as, train_l_as)
train_sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(random_seed))
val_dataset = TensorDataset(val_id_as, val_l_as)
test_dataset = TensorDataset(test_id_as, test_l_as)

batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [29]:
# train and test the data with best hyperparameter
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 3, 4, 5], num_filters=[100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding


.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:54<00:00, 7368.57it/s]


Start training...

Epoch  1 | Train Loss: 0.814914 | Train Acc: 0.5463 | Train F1-Macro: 0.4882
Epoch  1 | Validation Loss: 0.674852 | Validation Acc: 0.6064 | Validation F1-Macro: 0.4738
.........................................................................................
Epoch  2 | Train Loss: 0.824327 | Train Acc: 0.5326 | Train F1-Macro: 0.4750
Epoch  2 | Validation Loss: 0.693759 | Validation Acc: 0.5984 | Validation F1-Macro: 0.4846
.........................................................................................
Epoch  3 | Train Loss: 0.818627 | Train Acc: 0.5432 | Train F1-Macro: 0.4899
Epoch  3 | Validation Loss: 0.694404 | Validation Acc: 0.5984 | Validation F1-Macro: 0.4846
.........................................................................................
Epoch  4 | Train Loss: 0.810522 | Train Acc: 0.5441 | Train F1-Macro: 0.4935
Epoch  4 | Validation Loss: 0.693052 | Validation Acc: 0.5984 | Validation F1-Macro: 0.4688
...................................